In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from drone import compute_and_save_activations

In [3]:
import torch
from transformers import AutoModel

model = AutoModel.from_pretrained("microsoft/deberta-v3-base")

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, index):
        return {"input_ids": self.data[index], "attention_mask": torch.ones_like(self.data[index])}

    def __len__(self):
        return len(self.data)

In [5]:
data = torch.arange(50 * 32).reshape(50, 32)

dataset = TokenizedDataset(data)

In [6]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16)

In [7]:
compute_and_save_activations(model, dataloader, path_to_save="./", subsample=.3)

TypeError: 'NoneType' object is not subscriptable

In [11]:
model.encoder.get_rel_pos(torch.zeros(1, 7, 768))

tensor([[[ 0, -1, -2, -3, -4, -5, -6],
         [ 1,  0, -1, -2, -3, -4, -5],
         [ 2,  1,  0, -1, -2, -3, -4],
         [ 3,  2,  1,  0, -1, -2, -3],
         [ 4,  3,  2,  1,  0, -1, -2],
         [ 5,  4,  3,  2,  1,  0, -1],
         [ 6,  5,  4,  3,  2,  1,  0]]])

In [ ]:
!ls

In [ ]:
from drone import concat_activations_by_layers

concat_activations_by_layers("./")

In [ ]:
!ls

In [ ]:
!rm *.pth


In [ ]:
for batch in dataloader:
    break

with torch.inference_mode():
    output = model(batch, output_hidden_states=True)

    embeds = model.embeddings(batch)
    other_output = model.encoder(embeds, torch.ones_like(batch))

    print(torch.allclose(output.last_hidden_state, output.hidden_states[-1]))
    print(torch.allclose(output.hidden_states[0], embeds))

In [ ]:
model.encoder.layer[0].attention.self

In [ ]:
model.encoder.layer[0].attention.output

In [ ]:
model.encoder.layer[0].intermediate

In [ ]:
model.encoder.layer[0].output